#### Es 1: Eliminare un elemento ogni n
Eliminare tutti gli elementi con indice multiplo di "n" da una lista 

In [5]:
let drop n list = 
    (*Usiamo una funzione ausiliara riscorsiva,
      che accetta parametri aggiuntivi non presenti in drop:
      aux prende in input una lista e un contatore, che viene usato
      nelle chiamate ricorsive per tenere conto dell'indice dell'elemento 
      in testa alla lista.
    *)
    let rec aux i list = match list with 
        | [] -> []
        | h::t -> if i mod n = 0 (*n è un argomento di drop, quindi visibile in aux*)
                    then (aux (i+1) t) 
                    else h::(aux (i+1) t) in
    aux 1 list;;

let l = [1;2;3;4;5;6;];;
drop 3 l;;

val drop : int -> 'a list -> 'a list = <fun>


val l : int list = [1; 2; 3; 4; 5; 6]


- : int list = [1; 2; 4; 5]


#### Es 2: Slice
Dati due indici interi, i e k, lo slice è definito come la lista che contiene gli elementi tra l'i-esimo e il k-esimo della lista originale (inclusi i limiti i e k). Scrivere il programma che estrae lo slice da una lista

In [2]:
(*Definiamo slice come composizione di due funzioni, drop e take*)
let slice i k list = 
    (*drop n list scarta i primi n elementi da una lista*)
    let rec drop n list = match list with
        |[] -> []
        |x::t as l -> if n = 0 then l 
                     else drop (n-1) t
    in
    (*take n list prende i primi n elementi da una lista, 
      cioè scarta gli element dall'n+1-esimo in avanti*)
    let rec take n list = match list with 
        |[] -> []
        |x::t -> if n = 0 then []
                 else x::take (n-1) t
    in
    (*slice richiede di eseguire prima take e poi drop (o viceversa)
    Nel paradigma funzionale eseguire in sequenza due azioni 
    vuol dire comporne le funzioni*)
    drop i (take (k+1) list);;
    (*take (k-i+1) ( drop(i) list) è completamente equivalentemente*)


(*Le funzioni drop e take possono essere definite con la fold_l (o la fold_r)
    Poichè la funzione drop definita sopra è ricorsiva sia sull'argomento n che
    sull'argomento list, l'accumulatore della foldl dovrà contenere due informazioni:
    -list, il risultato che stiamo effettivamente costruendo, 
    -counter, per tener conto di quanti elementi sono ancora da scartare/prendere*)
let slice2 i k list = 
    let drop n list = 
        (*come accumulatore usiamo una coppia, quindi il risultato della foldl sarà
        una coppia (a' list * int). Il secondo valore è inutile, quindi non lo assegnamo
        a nessuna variabile*)
        let l, _ = List.fold_left (
            fun (list, counter) el -> if counter == 0 then (el::list, counter)
                                      else (list, counter-1)
        ) ([],n) list in
        (*usando (::) nella foldl, otteniamo e5::e4::e3::e2::e1, quindi la lista va invertita*)
        List.rev l 
    in
    let take n list = 
        let l, _ = List.fold_left (
            fun (list, counter) el -> if counter == 0 then (list, counter)
                                      else (el::list, counter-1)
        ) ([],n) list in
        List.rev l
    in
    drop i (take (k+1) list);;



(*La soluzione precedente percorre due volte tutta la lista, anche se lo splice è di pochi elementi
  Una alternativa, più specfica per questo problema, è definire una nuova versione della foldl, che 
  accumuli gli elementi di una lista solo fino all'n-esimo, e lasci il resto della lista inalterato*)
let slice3 i k list = 
    (*foldl_until n accumula solo i primi n elementi di una lista, restituendo una tupla come risultato:
      il primo valore è il risultato dell'accumulazione, il secondo è il resto della lista, non visitato *)
    let rec foldl_until n f acc list = 
        match list with
            |[] -> (acc, [])
            |x::t -> if n = 0 then (acc, x::t)
             else foldl_until (n-1) f (f acc x) t 
    in 
    (*sfruttando foldl_until, possiamo riscrivere 
        drop i (take (k+1) list)
      come: *)
    (*la funzione f da passare a alla foldl è più semplice di quella precedente, perchè
      non deve occuparsi del parametro n*)
    let taken, _ = foldl_until (k+1) (fun acc el -> el::acc) [] list in
    let taken = List.rev taken in (*questo let non è ricorsivo, quindi il "taken" a 
                                    destra dell'uguale è quello definito nella riga precedente*)
    (*per droppare i primi i elementi, basta conservare la seconda parte del risultato della 
      foldl_until, e scartare il primo. Poichè il risultato dell'accumulazione non ci interessa,
      come funzione accumulatrice possiamo mettere una funzione costante che restituisce sempre []*)
    let _, dropped = foldl_until i (fun _ _ -> []) [] taken in
    dropped;;



let l = [1;2;3;4;5;6;7;];;
slice 2 5 l;; 
slice2 2 5 l;;
slice3 2 5 l;;

val slice : int -> int -> 'a list -> 'a list = <fun>


val slice2 : int -> int -> 'a list -> 'a list = <fun>


val slice3 : int -> int -> 'a list -> 'a list = <fun>


val l : int list = [1; 2; 3; 4; 5; 6; 7]


- : int list = [3; 4; 5; 6]


- : int list = [3; 4; 5; 6]


- : int list = [3; 4; 5; 6]


#### Es 3: Replicate
Replicare gli elementi di una lista un dato numero di volte

In [3]:
(*definiamo una funzione prepend che aggiunge n copie
  dell'elemento x in testa a una lista*)
let rec prepend n x list =
    if n = 0 then list 
    else prepend (n-1) x (x :: list) ;;

(*usiamo una funzione ausiaria che, visitando la lista originale,
  costruisce il risultato dentro acc*)
let replicate list n =
    let rec aux acc = function
      | [] -> acc
      | h :: t -> aux (prepend n h acc) t in 
    aux [] (List.rev list);;

(*usando la foldr gli elementi di una lista vengono processati dall'ultimo al primo, 
  quindi con funzioni come (::) o prepend, aggiungiamo l'ultimo elemento in testa alla lista vuota,
  il penultimo in testa all'ultimo, il terzultimo in testa al penultimo e così via*)
let replicate2 list n = 
    (*essendo prepend una funzione int -> 'a -> 'a list -> 'a list,
      prepend n è esattamente la funzione f :('a ->'b -> 'b) che la 
      foldr si aspetta come parametro*)
    List.fold_right (prepend n) list [];;

let l = [1;2;3;4;];;
replicate l 3;;
replicate2 l 3;;

val prepend : int -> 'a -> 'a list -> 'a list = <fun>


val replicate : 'a list -> int -> 'a list = <fun>


val replicate2 : 'a list -> int -> 'a list = <fun>


val l : int list = [1; 2; 3; 4]


- : int list = [1; 1; 1; 2; 2; 2; 3; 3; 3; 4; 4; 4]


- : int list = [1; 1; 1; 2; 2; 2; 3; 3; 3; 4; 4; 4]


#### Es 4: Ordinare le liste per lunghezza
Supponiamo di considerare una lista i cui elementi sono essi stessi delle liste. Si scriva un programma che ordini gli elementi di questa lista in termini  della lunghezza delle liste da quella di lunghezza più piccola a quelle più grandi.

In [4]:
(*Implementiamo l'insertion-sort in maniera parametrica alla funzione cmp,
  così da poter riutilizzare lo stesso codice con diversi ordinamenti degli elementi*)

(*insert cmp e list inserisce e all'interno della lista ordinata list*)
let rec insert cmp e = function
    | [] -> [e]
    | h :: t as l -> if cmp e h <= 0 then e :: l else h :: insert cmp e t  
(*sort cmp list comincia a ordinare la lista dalla fine,
  inserendo ogni elemento all'interno del suffisso già ordinato*)
let rec sort cmp = function
    | [] -> []
    | h :: t -> insert cmp h (sort cmp t)
  
(* Associamo a ogni lista la sua lunghezza, usiamo le lunghezze per ordinare la lista,
  e rimuoviamo le lunghezze. Così facendo non effettuiamo chiamate ripetute a List.length*)
let length_sort lists =
   let lists = List.map (fun list -> List.length list, list) lists in
   let lists = sort (fun a b -> compare (fst a) (fst b)) lists in
   List.map snd lists;;

(* Esegue le stesse operazioni della funzione precedente, concatenando le operazioni 
  grazie all'operatore |>
  x |> f |> g è equivalente a g (f (x))*)
let length_sort2 lists = 
       lists 
    |> List.map (fun l -> (List.length l, l)) 
    |> sort (fun (l1, _) (l2, _) -> compare l1 l2) 
    |> List.map snd;;

val insert : ('a -> 'a -> int) -> 'a -> 'a list -> 'a list = <fun>


val sort : ('a -> 'a -> int) -> 'a list -> 'a list = <fun>


val length_sort : 'a list list -> 'a list list = <fun>


val length_sort2 : 'a list list -> 'a list list = <fun>
